# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 2        4.0  0.0  5.0  0.0  0.0  3.0  4.0  2.0  0.0  2.0  ...  0.0  2.0  0.0   
 3        3.0  0.0  4.0  0.0  0.0  3.0  2.0  2.0  0.0  5.0  ...  0.0  4.0  0.0   
 4        0.0  0.0  0.0  0.0  5.0  5.0  0.0  3.0  5.0  0.0  ...  0.0  4.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      0.0  0.0  5.0  0.0  0.0  0.0  4.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 296      4.0  0.0  4.0  0.0  0.0  4.0  3.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 297      0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  4.0  5.0  ...  0.0  0.0  4.0   
 298      0.0  0.0  5.0  4.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
Put all your implementation for your algorithm in the following cell only to handle the missing values

In [7]:
## Put all your implementation for your solution in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9

def nonzero_mean(matrix):
    '''
    A function to return a single vector of average values, ignoring values of 0
    '''
    return np.true_divide(matrix.sum(1), (matrix!=0).sum(1))

def row_similarity(row_a, row_b):
    '''
    A function to return the Pearson Correlation Coefficient of two rows in a given matrix
    '''
    
    # Filter each row by masking based on the other
    mask = row_a * row_b > 0
    
    if mask.sum() == 0:
        return 0
    
    # The rating vaulues/ intersection of each row
    intersection_a = row_a[mask]
    intersection_b = row_b[mask]
    
    # The average of each set of intersecting values
    mean_a = intersection_a.mean()
    mean_b = intersection_b.mean()
    
    # The normalised value for each set of values
    norm_a = intersection_a - mean_a
    norm_b = intersection_b - mean_b
    
    numerator = np.sum(norm_a * norm_b)
    denominator = np.sqrt(np.sum(np.square(norm_a))) * np.sqrt(np.sum(np.square(norm_b)))
    
    if denominator == 0:
        return 0
    
    return numerator/ denominator

def weighted_row_similarity(row_a, row_b, weight):
    '''
    A function to return the weighted Pearson Correlation Coefficient by reducing the impact of
    highly similar items or users
    '''
    return min(np.sum(row_a * row_b > 0), weight)/ weight * row_similarity(row_a, row_b)

def similarity_matrices(matrix):
    '''
    A function to return the Pearson Correlation Matrices for items and users, as well as the lists of 
    each users similar neighbours, and each items similar neigbours
    '''
    # Setting the sizes of the matrices based on the number of rows and columns of the input matrix
    user_similarity_matrix = np.zeros((len(matrix), len(matrix)))
    item_similarity_matrix = np.zeros((len(matrix.T), len(matrix.T)))

    # Instantiating a list of empty lists for each user and item in the given matrix
    similar_users = [[] for user in range(len(matrix))]
    similar_items = [[] for item in range(len(matrix.T))]
    
    # Iterating over each value of the empty matrices and filling them in, allocating the
    # user or item to their respective neighbour if they exceed the threshhold
    for (u, a), value in np.ndenumerate(user_similarity_matrix):
        user_similarity_matrix[u][a] = weighted_row_similarity(matrix[u], matrix[a], GAMMA)
        
        # Add the user_a to a list of user_u's similar neighbours if above the threshhold
        if a not in similar_users[u] and user_similarity_matrix[u][a] > ITA and u != a:
            similar_users[u].append(a)
        
    for (i, j), value in np.ndenumerate(item_similarity_matrix):
        item_similarity_matrix[i][j] = weighted_row_similarity(matrix.T[i], matrix.T[j], DELTA)
        
        # Add the item_j to a list of item_i's similar neighbours if above the threshhold
        if j not in similar_items[i] and item_similarity_matrix[i][j] > THETA and i != j:
            similar_items[i].append(j)
    
    return user_similarity_matrix, item_similarity_matrix, similar_users, similar_items
    
def predict(input_matrix):
    '''
    A function to predict the missing values of a given a sparsely populated user/ item matrix 
    '''    
    # Create a copy of each of the matrices required for better performance
    user_similarity_matrix, item_similarity_matrix, similar_user_ids, similar_item_ids = similarity_matrices(input_matrix)
    
    # Creating a copy of the input matrix 
    matrix = np.zeros_like(input_matrix)
    
    # Calculating the overall user and item averages for use in the final prediction
    overall_user_mean = nonzero_mean(input_matrix)
    overall_item_mean = nonzero_mean(input_matrix.T)
    
    for (user, item), value in np.ndenumerate(input_matrix):
        # Add the existing rating to the new matrix
        if value > 0:
            matrix[user][item] = value
            continue
        
        # Instantiating and resetting the numerator and denominator values, and the similar users and items for the prediction
        numerator_user = numerator_item = denominator_user = denominator_item = 0
        
        # Set the equations up for both user based and item based collaborative filtering
        similar_users = input_matrix[similar_user_ids[user]]
        similar_items = input_matrix[:,similar_item_ids[item]]
            
        # Calculate the mean vector for similar users and items, transposing the items
        user_mean = nonzero_mean(similar_users)
        item_mean = nonzero_mean(similar_items.T)
        
         # Selecting the similar items and users for this particular cell
        user_similarity = user_similarity_matrix[user, similar_user_ids[user]]
        item_similarity = item_similarity_matrix[item, similar_item_ids[item]]
        
        # Create a mask to only show similar users who have rated the item, or similar items that have been rated by the user
        users_mask = similar_users[:, item] > 0
        items_mask = similar_items[user, :] > 0
        
        # If the masks equate to 0, then there is no intersection, and the matrix value is 0
        if sum(users_mask) == 0 and sum(items_mask) == 0:
            continue
        
        # Calculate the values for user based collaborative filtering
        if sum(users_mask) > 0:
            numerator_user = np.sum(user_similarity[users_mask] * (similar_users[users_mask, item] - user_mean[users_mask]))
            denominator_user = np.sum(user_similarity[users_mask])

        # Calculate the values for item based collaborative filtering
        if sum(items_mask) > 0:
            numerator_item = np.sum(item_similarity[items_mask] * (similar_items[user, items_mask] - item_mean[items_mask]))
            denominator_item = np.sum(item_similarity[items_mask])
        
        # Logic for selecting which algoritm to use to predict values 
        if sum(users_mask) > 0 and sum(items_mask) > 0:
            prediction = LAMBDA * (overall_user_mean[user] + numerator_user/ denominator_user) + (1 - LAMBDA) * (overall_item_mean[item] + numerator_item/ denominator_item)
            matrix[user][item] = np.clip(prediction, 0, 5)
        
        elif sum(users_mask) > 0 and sum(items_mask) == 0:
            prediction = overall_user_mean[user] + numerator_user/ denominator_user
            matrix[user][item] = np.clip(prediction, 0, 5)        
        
        elif sum(users_mask) == 0 and sum(items_mask) > 0:
            prediction = overall_item_mean[item] + numerator_item/ denominator_item
            matrix[user][item] = np.clip(prediction, 0, 5)
            continue
        
    return matrix

# Finally, run the function in order to predict the missing values in place
imputed_train_ds = predict(imputed_train_ds)

# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.000000,0.000000,4.000000,4.000000,4.000000,0.0,0.0,4.000000,0.000000,0.000000,...,4.207039,0.000000,0.000000,0.000000,2.885749,0.00000,0.0,3.354167,0.000000,3.213436
1,4.277778,4.015908,0.000000,0.000000,5.000000,0.0,0.0,3.351413,0.000000,4.000000,...,0.000000,0.000000,0.000000,4.885593,0.000000,0.00000,0.0,0.000000,0.000000,4.284511
2,4.000000,4.019456,5.000000,0.000000,2.057959,3.0,4.0,2.000000,0.000000,2.000000,...,2.317104,2.000000,4.237185,4.130435,3.119954,2.20414,0.0,4.000000,0.000000,2.205190
3,3.000000,3.496341,4.000000,0.000000,3.313694,3.0,2.0,2.000000,0.000000,5.000000,...,3.211380,4.000000,2.600430,4.032762,2.885749,0.00000,0.0,0.000000,3.273345,3.010604
4,5.000000,3.758224,0.000000,0.000000,5.000000,5.0,0.0,3.000000,5.000000,0.000000,...,2.824403,4.000000,3.631579,0.000000,0.000000,0.00000,3.0,0.000000,3.000000,2.451837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,5.000000,3.232091,5.000000,0.000000,3.539843,0.0,4.0,3.829990,3.220022,4.000000,...,2.804060,0.000000,0.000000,4.885593,3.836204,0.00000,0.0,4.004070,3.273345,2.796148
296,4.000000,4.297455,4.000000,0.000000,3.713758,4.0,3.0,4.000000,4.003953,3.632812,...,3.559042,0.000000,0.000000,0.000000,2.797980,0.00000,0.0,4.632812,2.273345,2.822943
297,0.000000,0.000000,5.000000,0.000000,0.000000,0.0,0.0,5.000000,4.000000,5.000000,...,0.000000,0.000000,4.000000,0.000000,4.000000,2.00000,3.0,0.000000,0.000000,3.622720
298,3.792737,2.605562,5.000000,4.000000,3.574994,1.0,0.0,2.986467,1.003953,1.000000,...,1.623718,0.000000,2.849044,1.986467,3.797980,0.00000,0.0,3.410256,1.000000,2.991460


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[-0.18451593,  0.16509553,  0.01276922, ..., -0.14644561,
         0.08280928, -0.32642409],
       [ 0.24484502, -0.13062427,  0.55165814, ...,  0.10416129,
         0.15739639,  0.09910262],
       [ 0.32727518,  0.31179224,  0.535771  , ...,  0.53197119,
         0.36619927,  0.37103195],
       ...,
       [ 0.53744792,  0.51825493, -0.25409667, ...,  0.04305133,
         0.13968975, -0.06978167],
       [ 0.52488185,  0.4511469 ,  0.5497591 , ..., -0.03781263,
         0.47891473,  0.5673369 ],
       [-0.26740104,  0.36296889,  0.02646802, ..., -0.22515471,
         0.12482567,  0.50914809]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 2.16324495,
        0.        ],
       [0.        , 3.65942944, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [5.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.34654733, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7651535492390668, RMSE: 0.9887805181492954
